In [9]:
import os
from gpuutils import GpuUtils
GpuUtils.allocate(gpu_count=1, framework='keras')
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

GPU 2  will be allocated


In [2]:
import matplotlib.pyplot as plt
import pandas as pd
from creating_models import load_models

import plot_functions as pf
import data_manage as dm

import numpy as np
from tensorflow.keras.models import load_model

In [11]:
x_test, y_test, smask_test, signal, noise, std, mean = dm.load_data()

In [12]:
path = '/home/halin/Autoencoder/Models/CNN_101/CNN_101_model_5.h5'
save_path = '/home/halin/Autoencoder/Models/test_models'

In [ ]:
# pf.plot_performance(path, x_test, smask_test,save_path, std, mean)

In [ ]:
# model_names = ['CNN_102_model_5', 'CNN_102_model_7']
# results = dm.create_dataframe_of_results(model_names=model_names)
# plot_path='/home/halin/Autoencoder/Models/plots'
# pf.plot_table(plot_path)
# pf.noise_reduction_from_results(results=results,best_model='')

In [4]:
path = '/home/halin/Autoencoder/Models/Wrong trained models/CNN_003/CNN_003_model_9.h5'
path = 
model = load_model()
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
first_layer (InputLayer)     [(None, 100, 1)]          0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 100, 128)          512       
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 50, 128)           0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 256)               1638656   
_________________________________________________________________
dense_11 (Dense)             (None, 6400)              1644800   
_________________________________________________________________
reshape_5 (Reshape)          (None, 50, 128)           0   

In [6]:
model.get_layer(index=3)